In [1]:
import asyncio
import logging
from datetime import datetime

import influxdb_client
import pandas as pd
from IPython.display import Markdown, display

from tastytrade.common.logging import setup_logging
from tastytrade.config import RedisConfigManager
from tastytrade.config.enumerations import Channels
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.messaging.processors import (
    RedisEventProcessor,
    TelegrafHTTPEventProcessor,
)
from tastytrade.utils.time_series import forward_fill

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

2025-08-10 08:19:50 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20250810.log


# Service Connections

In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")

dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

influx_user = InfluxCredentials(config=config)
influxdb = influxdb_client.InfluxDBClient(
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)

# Safely attach processors only if the router is initialized
router = dxlink.router
if router is None:
    raise RuntimeError("DXLink router not initialized after open()")
handlers_dict = getattr(router, "handler", None)
if handlers_dict is None:
    raise RuntimeError("DXLink router.handler mapping not initialized")
for h in handlers_dict.values():
    h.add_processor(TelegrafHTTPEventProcessor())
    h.add_processor(RedisEventProcessor())

2025-08-10 08:19:53 - INFO:tastytrade.config.manager:174:Initialized 19 variables from .env file in Redis
2025-08-10 08:19:53 - DEBUG:asyncio:848:Get address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32>
2025-08-10 08:19:53 - DEBUG:asyncio:848:Get address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32>
2025-08-10 08:19:53 - DEBUG:asyncio:858:Getting address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32> took 29.803ms: [(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('170.76.244.142', 443))]
2025-08-10 08:19:53 - DEBUG:asyncio:858:Getting address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32> took 29.803ms: [(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('170.76.244.142', 443))]
2025-08-10 08:19:53 - DEBUG:asyncio:535:<asyncio.sslproto.SS

2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:225:SETUP
2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:UNAUTHORIZED
2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:UNAUTHORIZED
2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:AUTHORIZED
2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:AUTHORIZED
2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:1
2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:1
2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:3
2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:3
2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:5
2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:5
2025-08-10 08:19:53 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:7
2025-08-10 08:19:53 - INFO:tastytrade.messaging.han

# Market Data Subscriptions

In [3]:
start_time = datetime(2025, 8, 8)

symbols = ["BTC/USD:CXTALP", "NVDA", "AAPL", "QQQ", "SPY", "SPX"]  # , "/ESH25:XCME"]
# symbols = ["BTC/USD:CXTALP", "NVDA",]
# symbols = ["AAPL", "QQQ"]  #
# symbols = ["SPY", "SPX"]  #
intervals = ["1d", "1h", "30m", "15m", "5m", "m"]

# ticker subscriptions
await dxlink.subscribe(symbols)

# candle subscriptions
for symbol in symbols:
    for interval in intervals:
        coroutine = dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )
        await asyncio.wait_for(coroutine, timeout=10)

2025-08-10 08:19:55 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP
2025-08-10 08:19:55 - INFO:tastytrade.connections.sockets:236:Added subscription: NVDA
2025-08-10 08:19:55 - INFO:tastytrade.connections.sockets:236:Added subscription: AAPL
2025-08-10 08:19:55 - INFO:tastytrade.connections.sockets:236:Added subscription: NVDA
2025-08-10 08:19:55 - INFO:tastytrade.connections.sockets:236:Added subscription: AAPL
2025-08-10 08:19:55 - INFO:tastytrade.connections.sockets:236:Added subscription: QQQ
2025-08-10 08:19:55 - INFO:tastytrade.connections.sockets:236:Added subscription: SPY
2025-08-10 08:19:55 - INFO:tastytrade.connections.sockets:236:Added subscription: SPX
2025-08-10 08:19:55 - INFO:tastytrade.connections.sockets:236:Added subscription: QQQ
2025-08-10 08:19:55 - INFO:tastytrade.connections.sockets:236:Added subscription: SPY
2025-08-10 08:19:55 - INFO:tastytrade.connections.sockets:236:Added subscription: SPX
2025-08-10 08:19:55 - INFO:tastytrade.c

In [4]:
# forward fill
for symbol in symbols:
    for interval in intervals:
        event_symbol = f"{symbol}{{={interval}}}"
        logging.debug("Forward-filling %s", event_symbol)
        forward_fill(symbol=event_symbol, lookback_days=5)

2025-08-10 08:20:00 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-10 08:20:00 - INFO:root:124:Forward-fill added 1 events for BTC/USD:CXTALP{=15m}
2025-08-10 08:20:00 - INFO:root:124:Forward-fill added 1 events for BTC/USD:CXTALP{=15m}
2025-08-10 08:20:00 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-10 08:20:00 - INFO:root:124:Forward-fill added 9 events for BTC/USD:CXTALP{=5m}
2025-08-10 08:20:00 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-10 08:20:00 - INFO:root:124:Forward-fill added 9 events for BTC/USD:CXTALP{=5m}
2025-08-10 08:20:00 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-10 08:20:00 - INFO:root:124:Forward-fill added 58 events for BTC/USD:CXTALP{=m}
2025-08-10 08:20:00 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-10 08:20:00 - INFO:root:124:Forward-fill added 58 events for BTC/USD:CXTALP{=m}
2025-08-10 08:20:01 - INFO:root:120

# Check | Market Data feeds

In [5]:
symbol = "SPX{=m}"
symbol = "BTC/USD:CXTALP{=5m}"
# symbol = "/ESH25:XCME{=m}"

display(Markdown(f"**Candle Feed:** {symbol}"))
router = dxlink.router
if router is None or getattr(router, "handler", None) is None:
    raise RuntimeError("Router or handler map not initialized")
handlers_dict = router.handler
candle_handler = handlers_dict.get(Channels.Candle)
if candle_handler is None:
    raise KeyError("Candle channel handler not available")
feed_proc = candle_handler.processors.get("feed")
if feed_proc is None:
    raise KeyError("'feed' processor missing on Candle handler")
frames = getattr(feed_proc, "frames", {})
if symbol not in frames:
    raise KeyError(f"No frames for symbol {symbol}")
(frames[f"{symbol}"].tail(5).sort(by="time", descending=True))

**Candle Feed:** BTC/USD:CXTALP{=5m}

eventSymbol,time,eventFlags,index,sequence,count,open,high,low,close,volume,bidVolume,askVolume,openInterest,vwap,impVolatility
str,datetime[μs],i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BTC/USD:CXTALP{=5m}""",2025-08-10 08:20:00,0,7536868740562944000,0,20,117930.77,117935.94,117930.77,117935.94,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-10 08:15:00,0,7536867452072755200,0,1501,117874.72,117953.96,117860.24,117930.08,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-10 08:10:00,0,7536866163582566400,0,657,117751.16,117883.85,117749.01,117874.14,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-10 08:05:00,0,7536864875092377600,0,3891,117637.25,117771.67,117633.16,117750.58,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-10 08:00:00,0,7536863586602188800,0,2479,117777.45,117786.51,117627.27,117637.25,null,null,null,0.0,null,null


In [6]:
display(Markdown(f"**Quote Feed:** {symbol}"))
router = dxlink.router
if router is None or getattr(router, "handler", None) is None:
    raise RuntimeError("Router or handler map not initialized")
quote_handler = router.handler.get(Channels.Quote)
if quote_handler is None:
    raise KeyError("Quote channel handler not available")
feed_proc = quote_handler.processors.get("feed")
if feed_proc is None:
    raise KeyError("'feed' processor missing on Quote handler")
feed_proc.pl

**Quote Feed:** BTC/USD:CXTALP{=5m}

eventSymbol,bidPrice,askPrice,bidSize,askSize
str,f64,f64,f64,f64
"""SPY""",610.77,638.66,null,null
"""QQQ""",565.0,575.99,null,null
"""NVDA""",182.81,183.0,null,null
"""AAPL""",229.85,230.0,null,null
"""BTC/USD:CXTALP""",117543.69,118328.19,0.05,0.01
"""SPX""",6338.53,6434.76,null,null


In [ ]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [ ]:
await dxlink.close()